In [1]:
import pandas as pd
import numpy as np

## Chargement du fichier taux_interet

In [41]:
taux_interet = pd.read_csv('data/taux_interet.csv')

## Conversion de la colonne data en datetime

In [42]:
taux_interet['date']=pd.to_datetime(taux_interet['date'])

## Création de la colonne année_mois

In [43]:
taux_interet['année_mois'] = taux_interet['date'].dt.to_period('M')

## Tri du dataframe par date

In [44]:
taux_interet = taux_interet.sort_values(by='date')

## Chargement du fichier transactions

In [45]:
transaction = np.load('data/transactions.npz')

## Affichage du nom des colonnes et du nombre de lignes pour chaque colonnes

In [46]:
for key in transaction.files:
    # transaction[key] accède au tableau NumPy associé à la clé
    print(f"Taille de {key}: {transaction[key].shape}")

Taille de id_transaction: (8318280,)
Taille de date_transaction: (8318280,)
Taille de prix: (8318280,)
Taille de departement: (25038689,)
Taille de id_ville: (8318280,)
Taille de ville: (99664044,)
Taille de code_postal: (8318280,)
Taille de adresse: (167154686,)
Taille de type_batiment: (76761974,)
Taille de vefa: (8318280,)
Taille de n_pieces: (8318280,)
Taille de surface_habitable: (8318280,)
Taille de id_parcelle_cadastre: (124774199,)
Taille de latitude: (8318280,)
Taille de longitude: (8318280,)
Taille de surface_dependances: (29933185,)
Taille de surface_locaux_industriels: (25445296,)
Taille de surface_terrains_agricoles: (30594823,)
Taille de surface_terrains_sols: (46435763,)
Taille de surface_terrains_nature: (25869120,)


## Sélection des data significatives

In [47]:
colonnes_filtrees = {}

for key in transaction.files:
    if transaction[key].shape[0] == 8318280:  # Vérifier si le nombre de lignes correspond
        colonnes_filtrees[key] = transaction[key]

## Création du dataframe transactions à partir des colonnes filtrées

In [48]:
transactions = pd.DataFrame(colonnes_filtrees)
transactions.head()

,id_transaction,date_transaction,prix,id_ville,code_postal,vefa,n_pieces,surface_habitable,latitude,longitude
0,126289,2014-01-02,197000.0,427,1600,False,4,84,45.942301,4.770694
1,126606,2014-01-02,157500.0,451,1440,False,4,103,46.236407,5.262935
2,123875,2014-01-02,112000.0,365,1290,False,3,78,46.260087,4.918587
3,130652,2014-01-02,173020.0,202,1150,False,4,72,45.899056,5.354210
4,132775,2014-01-03,49023.3,27,1360,False,5,105,45.832127,5.097926


## Création de la colonne année_mois

In [49]:
transactions['année_mois'] = transactions['date_transaction'].dt.to_period('M')

## Ajout de la colonne taux au dataframe transaction à l'aide d'une jointure sur la colonne année_mois

In [50]:
transactions = pd.merge(transactions, taux_interet[['année_mois', 'taux']], on='année_mois', how='left')

In [51]:
transactions.head()

,id_transaction,date_transaction,prix,id_ville,code_postal,vefa,n_pieces,surface_habitable,latitude,longitude,année_mois,taux
0,126289,2014-01-02,197000.0,427,1600,False,4,84,45.942301,4.770694,2014-01,NaN
1,126606,2014-01-02,157500.0,451,1440,False,4,103,46.236407,5.262935,2014-01,NaN
2,123875,2014-01-02,112000.0,365,1290,False,3,78,46.260087,4.918587,2014-01,NaN
3,130652,2014-01-02,173020.0,202,1150,False,4,72,45.899056,5.354210,2014-01,NaN
4,132775,2014-01-03,49023.3,27,1360,False,5,105,45.832127,5.097926,2014-01,NaN


## Suppression des lignes où le taux n'apparait pas

In [52]:
print(transactions.shape)
print(transactions.isnull().sum())

(8318280, 12)
id_transaction            0
date_transaction          0
prix                      0
id_ville                  0
code_postal               0
vefa                      0
n_pieces                  0
surface_habitable         0
latitude                  0
longitude                 0
année_mois                0
taux                 575223
dtype: int64


In [53]:
transactions.dropna(inplace=True)

## Retraitement de la colonne code_postal

In [54]:
transactions['code_postal'] = transactions['code_postal'].astype(str) # conversion en chaine de caractères

In [55]:
transactions['code_postal'] = transactions['code_postal'].apply(lambda x: x.zfill(5)) # ajout d'un 0 s'il n'y a que 4 caractères

In [56]:
transactions.head()

,id_transaction,date_transaction,prix,id_ville,code_postal,vefa,n_pieces,surface_habitable,latitude,longitude,année_mois,taux
5199,130463,2014-12-01,200000.0,108,01270,False,5,115,46.386632,5.351503,2014-12,2.55
5200,131354,2014-12-01,230000.0,141,01300,False,4,94,45.824796,5.673966,2014-12,2.55
5201,130763,2014-12-01,118000.0,283,01100,False,3,90,46.261427,5.665573,2014-12,2.55
5202,128794,2014-12-01,929000.0,143,01220,False,6,159,46.348864,6.143717,2014-12,2.55
5203,132111,2014-12-01,227280.0,72,01250,False,4,82,46.180232,5.321355,2014-12,2.55


## Ajout de la colonne département

In [57]:
transactions['département'] = transactions['code_postal'].str.slice(0, 2)

In [58]:
transactions.head()

,id_transaction,date_transaction,prix,id_ville,code_postal,vefa,n_pieces,surface_habitable,latitude,longitude,année_mois,taux,département
5199,130463,2014-12-01,200000.0,108,01270,False,5,115,46.386632,5.351503,2014-12,2.55,01
5200,131354,2014-12-01,230000.0,141,01300,False,4,94,45.824796,5.673966,2014-12,2.55,01
5201,130763,2014-12-01,118000.0,283,01100,False,3,90,46.261427,5.665573,2014-12,2.55,01
5202,128794,2014-12-01,929000.0,143,01220,False,6,159,46.348864,6.143717,2014-12,2.55,01
5203,132111,2014-12-01,227280.0,72,01250,False,4,82,46.180232,5.321355,2014-12,2.55,01


## Ajout de la colonne Région

In [59]:
correspondance_df = pd.read_csv('data/correspondance_region.csv') # chargement de la table de correspondance

In [60]:
correspondance_df.head()

,département,Région
0,01,Auvergne-Rhône-Alpes
1,02,Hauts-de-France
2,03,Auvergne-Rhône-Alpes
3,04,Provence-Alpes-Côte d'Azur
4,05,Provence-Alpes-Côte d'Azur


In [61]:
departement_to_region = dict(zip(correspondance_df['département'], correspondance_df['Région']))
transactions['Région'] = transactions['département'].map(departement_to_region)

In [62]:
transactions.head()

,id_transaction,date_transaction,prix,id_ville,code_postal,vefa,n_pieces,surface_habitable,latitude,longitude,année_mois,taux,département,Région
5199,130463,2014-12-01,200000.0,108,01270,False,5,115,46.386632,5.351503,2014-12,2.55,01,Auvergne-Rhône-Alpes
5200,131354,2014-12-01,230000.0,141,01300,False,4,94,45.824796,5.673966,2014-12,2.55,01,Auvergne-Rhône-Alpes
5201,130763,2014-12-01,118000.0,283,01100,False,3,90,46.261427,5.665573,2014-12,2.55,01,Auvergne-Rhône-Alpes
5202,128794,2014-12-01,929000.0,143,01220,False,6,159,46.348864,6.143717,2014-12,2.55,01,Auvergne-Rhône-Alpes
5203,132111,2014-12-01,227280.0,72,01250,False,4,82,46.180232,5.321355,2014-12,2.55,01,Auvergne-Rhône-Alpes


## Ajout de la colonne prix_m2

In [63]:
transactions['prix_m2'] = transactions.prix / transactions.surface_habitable
transactions.head()

,id_transaction,date_transaction,prix,id_ville,code_postal,vefa,n_pieces,surface_habitable,latitude,longitude,année_mois,taux,département,Région,prix_m2
5199,130463,2014-12-01,200000.0,108,01270,False,5,115,46.386632,5.351503,2014-12,2.55,01,Auvergne-Rhône-Alpes,1739.130435
5200,131354,2014-12-01,230000.0,141,01300,False,4,94,45.824796,5.673966,2014-12,2.55,01,Auvergne-Rhône-Alpes,2446.808511
5201,130763,2014-12-01,118000.0,283,01100,False,3,90,46.261427,5.665573,2014-12,2.55,01,Auvergne-Rhône-Alpes,1311.111111
5202,128794,2014-12-01,929000.0,143,01220,False,6,159,46.348864,6.143717,2014-12,2.55,01,Auvergne-Rhône-Alpes,5842.767296
5203,132111,2014-12-01,227280.0,72,01250,False,4,82,46.180232,5.321355,2014-12,2.55,01,Auvergne-Rhône-Alpes,2771.707317


## Suppréssion de lignes incohérentes

In [69]:
transactions=transactions[transactions['prix']>transactions['prix_m2']]

## Regroupement du dataframe par date et conservation des colonnes utiles à l'analyse

In [70]:
grouped = transactions.groupby(['année_mois', 'Région']).agg(
    nombre_transactions=('id_transaction', 'count'),
    taux_moyen=('taux', 'mean'),
    prix_m2=('prix_m2', 'mean')
).reset_index()

In [71]:
grouped.head()

,année_mois,Région,nombre_transactions,taux_moyen,prix_m2
0,2014-12,Auvergne-Rhône-Alpes,8689,2.55,2539.253819
1,2014-12,Bourgogne-Franche-Comté,2733,2.55,1614.456836
2,2014-12,Bretagne,3536,2.55,1923.711200
3,2014-12,Centre-Val de Loire,2550,2.55,1761.444114
4,2014-12,Grand Est,2264,2.55,1547.575417


## Modification du nom de la colonne année_mois en date_transaction

In [72]:
grouped = grouped.rename(columns={'année_mois': 'date_transaction'})

In [73]:
grouped.to_csv('transactions_csv.csv')